In [1]:
import pandas as pd
import numpy as np

### GWAS simulation population summary datasets

In [2]:
bbj_path = "/wynton/scratch/BMI206_NIC/noahs_project/simulations/bbj_sim_pop.tsv"
bbj = pd.read_csv(bbj_path, sep="\t")

In [3]:
page_path = "/wynton/scratch/BMI206_NIC/noahs_project/simulations/page_sim_pop.tsv"
page = pd.read_csv(page_path, sep="\t")

In [5]:
# bbj_path = "/wynton/scratch/BMI206_NIC/noahs_project/simulations/sep/bbj_only_pop.tsv"
# bbj_only = pd.read_csv(bbj_path, sep="\t")

In [6]:
# bbj_path = "/wynton/scratch/BMI206_NIC/noahs_project/simulations/sep/bbj_share_pop.tsv"
# bbj_share = pd.read_csv(bbj_path, sep="\t")

In [7]:
# page_path = "/wynton/scratch/BMI206_NIC/noahs_project/simulations/sep/page_only_pop.tsv"
# page_only = pd.read_csv(page_path, sep="\t")

In [8]:
# page_path = "/wynton/scratch/BMI206_NIC/noahs_project/simulations/sep/page_share_pop.tsv"
# page_share = pd.read_csv(page_path, sep="\t")

In [9]:
# read in the intersect SNPs as list
interset = pd.read_csv('interset_2.csv')['SNP']

### Check the size of variables

In [10]:
import sys

# Get the size of the variable in bytes
size_in_bytes = sys.getsizeof(bbj)

# Convert bytes to gigabytes
size_in_gb = size_in_bytes / (1024 ** 3)

print(f"The size of the variable is: {size_in_gb:.9f} GB")

The size of the variable is: 0.128626855 GB


In [13]:
page.head()

,SNP,A1,A2,BETA,SE
0,rs3934834,C,T,0.023554,0.673728
1,rs9442372,A,G,-0.104435,0.534401
2,rs6687776,C,T,-0.682486,0.574653
3,rs4970405,A,G,1.627117,0.902432
4,rs12726255,A,G,-0.013162,0.579338


In [14]:
# percent that has overlap with the different populations
len(interset)/bbj.shape[0]*100

50.0

# Simulate populations

In [23]:
def sim_function(df):
    # Vectorized simulation of new beta values
    df['BETA'] = np.random.normal(loc=df['BETA'], scale=df['SE'])

    # Ensuring BETA is second to last and SE is last
    cols_order = [col for col in df.columns if col not in ['BETA', 'SE']] + ['BETA', 'SE']
    df = df[cols_order]
    return df


def sim_betas(dfA, dfB, interset, ratio=0, random_state=42):
    # Check if 'BETA' and 'SE' columns exist in both DataFrames
    for df in [dfA, dfB]:
        if 'BETA' not in df.columns or 'SE' not in df.columns:
            raise ValueError("Required columns 'BETA' or 'SE' are missing in the DataFrame")
    
    # note ratio refers to the ratio of the entire sample being simulated and is thus doubled
    num_sim = round(len(interset) * ratio * 2)

    # Select the indices that I want to simulate on
    np.random.seed(random_state)
    indices = np.random.choice(len(interset), num_sim, replace=False)
    # print(len(indices))
    rsIDs = interset[indices]
    # print(rsIDs)

    # df of the yes selection and the no selection for which are simulated
    df_y_sel = dfB[dfB['SNP'].isin(rsIDs)].copy()
    df_n_sel = dfA[~dfA['SNP'].isin(rsIDs)].copy()
    
    # simulate for the replaced and the non-replaced
    df_y_sel = sim_function(df_y_sel)
    df_n_sel = sim_function(df_n_sel)

    # Concatenate the DataFrames
    df_out = pd.concat([df_y_sel, df_n_sel], axis=0, ignore_index=True)

    return df_out

In [28]:
# run the simulations and save the data
def run_sims(ratios, datasets, data_names, interset, random_states):
    if len(datasets) != len(data_names):
        raise ValueError("datasets and data_names must be the same length")
    
    for data_index in range(0, len(datasets)):
        for ratio in ratios:
            for rand in random_states:
                sim_data = sim_betas(datasets[data_index], datasets[data_index-1], interset, ratio, rand)
                file_name = f'simulations/out/{data_names[data_index]}_{ratio}_r{rand}.tsv'
                print(file_name)
                sim_data.to_csv(file_name, sep='\t', index=False, encoding='utf-8')

In [29]:
ratios = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5]
random_states = [1, 2, 3, 4, 5]

run_sims(ratios, [bbj, page], ['bbj', 'page'], interset, random_states)

simulations/out/bbj_0.01_r1.tsv
simulations/out/bbj_0.01_r2.tsv
simulations/out/bbj_0.01_r3.tsv
simulations/out/bbj_0.01_r4.tsv
simulations/out/bbj_0.01_r5.tsv
simulations/out/bbj_0.05_r1.tsv
simulations/out/bbj_0.05_r2.tsv
simulations/out/bbj_0.05_r3.tsv
simulations/out/bbj_0.05_r4.tsv
simulations/out/bbj_0.05_r5.tsv
simulations/out/bbj_0.1_r1.tsv
simulations/out/bbj_0.1_r2.tsv
simulations/out/bbj_0.1_r3.tsv
simulations/out/bbj_0.1_r4.tsv
simulations/out/bbj_0.1_r5.tsv
simulations/out/bbj_0.15_r1.tsv
simulations/out/bbj_0.15_r2.tsv
simulations/out/bbj_0.15_r3.tsv
simulations/out/bbj_0.15_r4.tsv
simulations/out/bbj_0.15_r5.tsv
simulations/out/bbj_0.2_r1.tsv
simulations/out/bbj_0.2_r2.tsv
simulations/out/bbj_0.2_r3.tsv
simulations/out/bbj_0.2_r4.tsv
simulations/out/bbj_0.2_r5.tsv
simulations/out/bbj_0.25_r1.tsv
simulations/out/bbj_0.25_r2.tsv
simulations/out/bbj_0.25_r3.tsv
simulations/out/bbj_0.25_r4.tsv
simulations/out/bbj_0.25_r5.tsv
simulations/out/bbj_0.3_r1.tsv
simulations/out/bbj